This is a script for simulating arbitrary activity and export its results.

In [1]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication


Bad key "nbagg.transparent" on line 426 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_path" on line 509 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_args" on line 512 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/

In [2]:
comm = UnityCommunication()

Select scene number. 
Please see https://github.com/xavierpuigf/virtualhome/tree/master/simulation#environment .

If you want to use the existing activity data provided by VirtualHome, you can load the dataset.
Please see the below link for getting the dataset. (Note that there are many errors in the dataset.)
https://github.com/xavierpuigf/virtualhome#dataset

In [3]:
scene = 1
scene_graph = "TrimmedTestScene" + str(scene) + "_graph"
executable_program_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/*/*.txt"
executable_program_list = []
for file_path in glob.glob(executable_program_path):
    executable_program_list.append(file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/", ""))

The HomeOntology is used for randomly selecting an activity.
Load HomeOntology as RDF graph.
If you do not select an activity randomly, you do not have to use the HomeOntology.

In [4]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=N771a6e0c60f0421e9750b683db4dee11 (<class 'rdflib.graph.Graph'>)>

In [5]:
def get_activity_from_ontology(activity_type):
    results = []
    qres = rdf_g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?activity where { 
    ?activity rdfs:subClassOf :%s .
 } 
       """ % activity_type)

    result = ""
    for row in qres:
        activity = "".join(row).replace("http://www.owl-ontologies.com/VirtualHome.owl#","")
        arr = activity.split("_")
        arr[0] = arr[0].capitalize()
        activity = " ".join(arr)
        result = activity
        results.append(result)
    return results

In [6]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [7]:
def get_activity_program(category):
    #unexecutable = ["Take shower", "Take shoes off", "Wash teeth", "Wash face", "Dust", "Clean toilet", "Clean room", "Scrubbing living room tile floor is once week activity for me", "Clean mirror", "Play games", "Play on laptop", "Read on sofa"]
    unexecutable = []
    executable = []
    activities = get_activity_from_ontology(category)
    for activity_name in activities:
        if activity_name in unexecutable:
            continue
        results = [program for program in program_list if program["name"] == activity_name]
        if len(results) == 0:
            print("Nothing: " + activity_name)
        else:
            print("Success: " + activity_name)
            executable.append({"activity_name": activity_name, "results": results})
    return executable

In [8]:
data_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/*/*.txt"
program_list = []
for file_path in glob.glob(data_path):
    file_name = file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/", "")
    if file_name in executable_program_list:
        program_name, description, list_of_steps = generate_list_of_steps(file_path)
        program_list.append({
            "file_name":file_name,
            "name": program_name,
            "description": description,
            "list_of_steps": list_of_steps
        })

In [9]:
executable_activity_list = []
for activity_class in ["BedTimeSleep", "EatingDrinking", "FoodPreparation", "GettingReady", "HouseArrangement", "HouseCleaning", "HygieneStyling", "Leisure", "Other", "PhysicalActivity", "SocialInteraction", "Work"]:
    executable_activity_list.extend(get_activity_program(activity_class))

Nothing: Wake me up
Nothing: Get ready for bed
Success: Sleep
Nothing: Put them to sleep
Nothing: Put alarm clock in bedroom
Success: Take nap
Nothing: Go to bed
Nothing: Read them bedtime story
Success: Go to sleep
Nothing: Bed time
Nothing: Get something to drink
Success: Eat dinner
Nothing: Eat cereal
Nothing: Eat meals
Nothing: Have dinner
Success: Eat
Nothing: Dining
Nothing: Eat snacks and drink tea
Nothing: Have ice coffee
Success: Drink
Nothing: Dance for table as we eat
Nothing: Eat while watching tv
Nothing: Eat dessert
Nothing: Eat family meals
Nothing: Eat cheese
Nothing: Eat ice cream
Nothing: Eat breaksfast
Nothing: Eat donuts
Success: Have snack
Nothing: Eat on fancy china
Nothing: Squeeze lemon
Success: Chop vegetables
Success: Make toast
Success: Toast bread
Success: Cook some food
Nothing: Make iced coffee
Success: Prepare pot of boiling water
Nothing: Make cereal
Success: Make coffee
Nothing: Cut steak
Nothing: Prepare sandwitch
Nothing: Bake
Nothing: Fix sandwhich
N

In [10]:
unsupport_unity_exec_time = {
    "PutOn": 10.0,
    "PutOff": 10.0,
    "Drop": 2.0,
    "Read": 1800.0,
    "Lie": 5.0,
    "Pour": 5.0,
    "Type": 10.0,
    "Watch": 7200.0,
    "Wash": 10.0,
    "Squeeze": 5.0,
    "PlugIn": 5.0,
    "PlugOut": 5.0,
    "Sleep": 21600.0,
    "Wake": 5.0
}

In [11]:
def check_unsupport_action(script):
    flag = True
    for line in script:
        m = re.search(r'\[.+\]', line)
        action = m.group().replace('[', '')
        action = action.replace(']', '')
        if action in [x for x in unsupport_unity_exec_time.keys()]:
            flag = False
            break
    return flag

In [12]:
def switchOff(g, class_names):
    for class_name in class_names:
        objects = find_nodes(g, class_name=class_name)
        for obj in objects:
            states = [x for x in obj["states"] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            obj["states"] = states
#             for node in g["nodes"]:
#                 if node["id"] == obj["id"]:
#                     node = obj
        return g

In [13]:
def export(activity_name, graph_state_list, activity_cnt, time_list):
    os.mkdir("graph_state_list_with_bbox_png/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt)
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list_with_bbox_png/scene" + str(scene) + "/"  + activity_name + "/" + activity_cnt + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list_with_bbox_png/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/activityList-program.txt", 'w') as f:
        for s in executed_program:
            f.write("%s\n" % s)

#     with open("graph_state_list_with_bbox_png/test_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/program-description.txt", 'w') as f:
#         f.write("%s\n" % activity["name"])
#         f.write("%s\n" % activity["description"])
        
    time_list = [str(time) for time in time_list]
    duration = "\n".join(time_list)
    with open("graph_state_list_with_bbox_png/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/duration.txt", 'w') as f:
        f.write(duration)

In [14]:
def update_bbox(pre_graph, current_graph):
    try:
        new_graph= copy.deepcopy(current_graph)
        
        for pre_node in pre_graph["nodes"]:
            for new_node in new_graph["nodes"]:
                if new_node["id"] == pre_node["id"]:
                    new_node["bounding_box"] = pre_node["bounding_box"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [15]:
def update_states(pre_graph, graph_state, object_id_map):
    try:
        new_graph= copy.deepcopy(pre_graph)
        
        skip = [object_id_map[x] for x in object_id_map]
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            if new_node_id in skip:
                continue
            temp_node_id = copy.deepcopy(new_node_id)
            # scriptで使用したobject_idとgraph_stateで反映されたidが異なる場合
            if str(new_node_id) in object_id_map:
                temp_node_id = object_id_map[str(new_node_id)]
            for graph_state_node in graph_state["nodes"]:
                if temp_node_id == graph_state_node["id"]:
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["states"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [16]:
def create_object_id_map(original_script, executed_program, g):
    try:
        object_id_map = {}
        object_name_map = {}
        for original_block_str, executed_block_str in zip(original_script, executed_program):
            original_action, original_obj_names_corr, original_inst_nums_corr = augmentation_utils.parseStrBlock(original_block_str)
            executed_action, executed_obj_names_corr, executed_inst_nums_corr = augmentation_utils.parseStrBlock(executed_block_str)
            
            #object_idのmapを作る
            for original_id, executed_id in zip(original_inst_nums_corr, executed_inst_nums_corr):
                if original_id not in object_id_map:
                    if original_id != executed_id:
                        object_id_map[original_id] = int(executed_id)
            
            #object_nameのmap作る
            for original_obj_name, original_id in zip(original_obj_names_corr, original_inst_nums_corr):
                g_objects = [x for x in g["nodes"] if x["id"] == original_id]
                if len(g_objects) != 0:
                    g_obj = g_objects[0]
                    if original_obj_name != g_obj["class_name"]:
                        object_name_map[original_obj_name] = g_obj["class_name"]
    except Exception as e:
        print(e)
    return object_id_map, object_name_map

In [17]:
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [19]:
for executable_activity in executable_activity_list:
    activity_list = executable_activity["results"]
    activity_name = executable_activity["activity_name"]
    activity_cnt = 0
    try:
        # if activity_name != "Relax on sofa": 
        #     continue
        os.mkdir("graph_state_list_with_bbox_png/scene" + str(scene) + "/" + activity_name)
        for activity in activity_list:
            if activity_cnt > 0:
                break
            try:
                new_graph_state_list = []
                time_list = []
                comm.reset(scene-1)
                comm.add_character('chars/Female2')
                success, g = comm.environment_graph();
                print(activity["file_name"])
                script = activity["list_of_steps"]
                executable_program = []
                for x in script:
                    executable_program.append("<char0> " + x)
                executable_program

                # check if supported action
                if check_unsupport_action(script) == False:
                    continue
                
                preconds = add_preconds.get_preconds_script(script).printCondsJSON()
                
                print(preconds)
                print("switch off")
                g = switchOff(g, ["computer", "tv", "faucet", "television"])
                
                g = check_programs.translate_graph_dict_nofile(g)
                info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
                message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
                # if delete randomly placced objects 
                graph_state_list = delete2000(graph_state_list)
                print(message)
                executed_program = []
                for x in executed_script:
                    executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

                if message != "Script is executable":
                    continue
                
                new_script = []
                for x in script:
                    new_script.append("<char0> " + x)
                    
                print(new_script)
                    
                i = 0
                
                object_id_map, object_name_map = create_object_id_map(new_script, executed_program, g)
                print("object_id_map")
                print(object_id_map)
                
                success, g = comm.environment_graph()
                g = update_states(g, graph_state_list[i], object_id_map)
                new_graph_state_list.append(copy.deepcopy(g))

                for instruction in executed_program:
                    i += 1
                    start = time.time()
                    success, message = comm.render_script(script=[instruction],
                                              #processing_time_limit=80,
                                              find_solution=False,
                                              image_width=480,
                                              image_height=360,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix=activity_name + str(activity_cnt),
                                              output_folder="Output/",
                                              frame_rate=20,
                                              vis_check_object=True, 
                                              vis_check_character=True, 
                                              out_graph=False
                                             )
                    time_list.append(time.time() - start)
                    success, g = comm.environment_graph()
                    print(str(success) + ": " + str(instruction))
                    g = update_states(g, graph_state_list[i], object_id_map)
                    new_graph_state_list.append(copy.deepcopy(g))

                if os.path.exists("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name) == False:
                    os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name)
                export(activity_name, new_graph_state_list, time_list, new_script, object_name_map,  out_dir_name)
                
                activity_cnt += 1
            except Exception as e:
                print(e.args)
    except Exception as e:
        print(e.args)

(17, 'File exists')
results_text_rebuttal_specialparsed_programs_upworknturk_second/split141_3.txt
results_intentions_march-13-18/file509_2.txt
results_intentions_march-13-18/file1027_2.txt
results_intentions_march-13-18/file140_2.txt
results_intentions_march-13-18/file244_2.txt
results_intentions_march-13-18/file181_1.txt
results_intentions_march-13-18/file736_1.txt
results_intentions_march-13-18/file269_2.txt
results_intentions_march-13-18/file875_1.txt
results_intentions_march-13-18/file274_1.txt
results_text_rebuttal_specialparsed_programs_turk_third/split4_3.txt
[{'atreach': [['chair', '1'], ['table', '1']]}, {'atreach': [['food_food', '1'], ['plate', '1']]}, {'atreach': [['food_food', '2'], ['plate', '1']]}, {'atreach': [['fork', '1'], ['plate', '1']]}, {'atreach': [['plate', '1'], ['table', '1']]}, {'location': [['plate', '1'], ['kitchen', '1']]}, {'location': [['plate', '1'], ['dining_room', '1']]}, {'location': [['food_food', '1'], ['kitchen', '1']]}, {'location': [['food_food

("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fea602aec10>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fea600c3a30>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa8095250>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa8095310>: Failed to establish a new connection: [Errno 61] Connection refused'))

("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa80a3fd0>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa8095580>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa8095340>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fea794e8430>: Failed to establish a new connection: [Errno 61] Connection refused'))

("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa80b2ee0>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fea6029fe80>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fea98413fa0>: Failed to establish a new connection: [Errno 61] Connection refused'))",)
("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feaa80954c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
 """
# for executable_activity in executable_activity_list:
#     activity_list = executable_activity["results"]
#     activity_name = executable_activity["activity_name"]
activity_cnt = 0
activity_name = "test2"
try:
    # if activity_name != "Relax on sofa": 
    #     continue
    os.mkdir("graph_state_list_with_bbox_png/test_scene" + str(scene) + "/" + activity_name)
#     for activity in activity_list:
#     if activity_cnt > 0:
#         break
    try:
        new_graph_state_list = []
        time_list = []
        comm.reset(scene-1)
        comm.add_character('chars/Female2')
        success, g = comm.environment_graph();
#         print(activity["file_name"])
#         script = activity["list_of_steps"]

        #swo056
        script = [
            "[WALK] <home_office> (336)", 
            "[WALK] <drawing> (417)"
#             "[WATCH] <drawing> (417)"
                 ]

        executable_program = []
        for x in script:
            executable_program.append("<char0> " + x)
        executable_program

#         # check if supported action
#         if check_unsupport_action(script) == False:
#             continue

        preconds = add_preconds.get_preconds_script(script).printCondsJSON()
        g = check_programs.translate_graph_dict_nofile(g)
        info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
        message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
        # if delete randomly placced objects 
#         graph_state_list = delete2000(graph_state_list)
        print(message)
        executed_program = []
        for x in executed_script:
            executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

#         if message != "Script is executable":
#             continue

        i = 0
        comm.expand_scene(graph_state_list[0])
        success, g = comm.environment_graph()
        g = update_bbox(g, graph_state_list[i])
        new_graph_state_list.append(copy.deepcopy(g))

        for instruction in executed_program:
            i += 1
            start = time.time()
            success, message = comm.render_script(script=[instruction],
                                      #processing_time_limit=80,
                                      find_solution=False,
                                      image_width=480,
                                      image_height=360,  
                                      #skip_animation=False,
                                      recording=True,
#                                               gen_vid=True,
                                      #save_scene_states=True,
                                      file_name_prefix=activity_name + str(activity_cnt),
                                      output_folder="Output/",
                                      frame_rate=15
                                     )
            time_list.append(time.time() - start)
            success, g = comm.environment_graph()
            print(str(success) + ": " + str(instruction))
            g = update_bbox(g, graph_state_list[i])
            new_graph_state_list.append(copy.deepcopy(g))

        export(activity_name, new_graph_state_list, str(activity_cnt), time_list)
#         with open("Output/" + activity_name + activity_cnt + "/" + str(scene) + "/ftaa_" + activity_name + activity_cnt + ".txt")  as f:
#             line = f.readline()
#             line.split(" ")
        activity_cnt += 1
    except Exception as e:
        print(e.args)
except Exception as e:
    print(e.args)
"""